In [1]:
from google.colab import drive

drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2 # 학교에 없는 코드
import numpy as np
import pandas as pd
from PIL import Image
from datetime import datetime
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary # 학교에 없는 코드

from torchvision import transforms
from torchvision.models import resnet34, resnet50

In [3]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}

        try:
            with open(image_ids, 'r') as f:
                reader = csv.reader(f)
                next(reader) # csv 파일 맨 윗줄 생략
                for row in reader:
                    self.labels[int(row[0])] = list(map(int, row[1:]))
        except Exception as err:
            raise err

        self.image_ids = list(self.labels.keys())

    def __len__(self): # 스페셜 메소드
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]: # 스페셜 메소드 -> 데이터 레이블과 같이 가져옴
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

In [4]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [5]:
data_dir = '/content/gdrive/MyDrive/data/dirty mnist/'
train_dir = os.path.join(data_dir, 'train_dirty_mnist')
train_label_path = os.path.join(data_dir, 'dirty_mnist_answer.csv')
test_dir = os.path.join(data_dir, 'test_dirty_mnist')
test_label_path = os.path.join(data_dir, 'sample_submission.csv')

try:
    os.chdir(data_dir)
    print('current_dir:', os.getcwd())
except Exception as err:
    print(str(err))

current_dir: /content/gdrive/MyDrive/data/dirty mnist


In [6]:
try:
    print('current dir:', os.getcwd())
    trainset = MnistDataset('./train_dirty_mnist', './dirty_mnist_answer.csv',
                            transforms_train) # pixel, label, transform
    testset = MnistDataset('./test_dirty_mnist', './sample_submission.csv',
                           transforms_test) # pixel, label, transform

    train_loader = DataLoader(trainset, batch_size=32, num_workers=2)
    test_loader = DataLoader(testset, batch_size=8, num_workers=1)
except Exception as err:
    print(str(err))

current dir: /content/gdrive/MyDrive/data/dirty mnist


In [7]:
class Resnet34(nn.Module):
    def __init__(self) -> None:
        super.__init__()
        self.resnet = resnet34(pretrained=True)
        self.classifier = nn.Linear(1000, 26)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x


class Resnet50(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, 26)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)
model = Resnet34().to(device)
print(summary(model, input_size=(3, 256, 256)))

device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           9,408
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]           4,096
       BatchNorm2d-6           [-1, 64, 64, 64]             128
              ReLU-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
             ReLU-10           [-1, 64, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]          16,384
      BatchNorm2d-12          [-1, 256, 64, 64]             512
           Conv2d-13          [-1, 256, 64, 64]          16,384
      BatchNorm2d-14      

In [8]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()

loss_list = []
acc_list = []

num_epochs = 1
model.train()
start_time = datetime.now()
print('======================= start training =======================')
for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images).to(device)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        if (i+1) % (int(0.1*len(train_loader))) == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            loss_list.append(loss)
            acc_list.append(acc)
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item():.5f}, acc: {acc.item():.5f}')

end_time = datetime.now()
print('\nElapsed Time:', end_time - start_time)
print('======================== end training ========================')

======================= start training =======================
epoch: 0, step: 155, loss: 0.69822, acc: 0.52163
epoch: 0, step: 311, loss: 0.68661, acc: 0.53606
epoch: 0, step: 467, loss: 0.68989, acc: 0.56010
epoch: 0, step: 623, loss: 0.68627, acc: 0.53365
epoch: 0, step: 779, loss: 0.67931, acc: 0.53846
epoch: 0, step: 935, loss: 0.67833, acc: 0.54808
epoch: 0, step: 1091, loss: 0.67192, acc: 0.57212
epoch: 0, step: 1247, loss: 0.67575, acc: 0.54808
epoch: 0, step: 1403, loss: 0.67221, acc: 0.55889
epoch: 0, step: 1559, loss: 0.67410, acc: 0.54688
epoch: 1, step: 155, loss: 0.66548, acc: 0.54567
epoch: 1, step: 311, loss: 0.66395, acc: 0.57812
epoch: 1, step: 467, loss: 0.62714, acc: 0.60096
epoch: 1, step: 623, loss: 0.64461, acc: 0.57812
epoch: 1, step: 779, loss: 0.62761, acc: 0.60096
epoch: 1, step: 935, loss: 0.63642, acc: 0.60697
epoch: 1, step: 1091, loss: 0.60984, acc: 0.62861
epoch: 1, step: 1247, loss: 0.62943, acc: 0.60216
epoch: 1, step: 1403, loss: 0.62357, acc: 0.61178

In [ ]:
plt.title('acc trend')
plt.xlabel('epochs')

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images).to(device)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()

submit.to_csv('./submit.csv', index=False)